In [1]:
import sys
import tensorflow as tf
sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/")

# import customized modules
import preprocess, data_generator, check_result, metrics, loss, train

sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid")
import pyramid


# Insert Path
# {model_name}_{epoch}ep_{model_inpusize}_{Generator | aG/fG/sG}
FILE_NAME = 'pyramid_30ep_512_sG_wo_decay'
INPUT_SHAPE = (512, 512, 1)
RESIZE_SHAPE = (1024, 1024, 1)
MODEL_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/" + FILE_NAME + ".tf"
HISTORY_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/history/" + FILE_NAME + '_history' ".json"

TRAIN_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Train'
TEST_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Test'

2024-01-31 08:51:15.693635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 08:51:15.693660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 08:51:15.694722: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 08:51:15.700091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 08:51:16.277156: W tensorflow/compiler/tf2

In [2]:
# Define custom objects for loading the model
custom_objects = {'DiceLoss': loss.DiceLoss(), 
                  'sensitivity': metrics.sensitivity,
                  'specificity': metrics.specificity,
                  'accuracy' : metrics.accuracy}


# Create augmentation
train_preproc = preprocess.build_augmentation_for_general(RESIZE_SHAPE[0], RESIZE_SHAPE[0])
test_preproc = preprocess.build_augmentation_for_general(RESIZE_SHAPE[0], RESIZE_SHAPE[0], is_train=False)

# Create train, test generator
train_generator = data_generator.SlicedDataGenerator(
    TRAIN_SOURCE, 
    number_of_images = 1,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    resize_shape = RESIZE_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

test_generator = data_generator.SlicedDataGenerator(
    TRAIN_SOURCE,
    number_of_images = 1,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    resize_shape = RESIZE_SHAPE,
    is_train=False,
    augmentation= test_preproc
)

# Create model
model = pyramid.build_model(INPUT_SHAPE)
model.summary()

2024-01-31 08:51:17.435620: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 08:51:17.471108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 08:51:17.471322: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 512, 512, 16)         32        ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 8)          16        ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 512, 512, 16)         160       ['input_1[0][0]']             
                                                                                              

In [3]:
train.model_train(model,
                  epoch = 30,
                  train_generator = train_generator,
                  test_generator = test_generator,
                  model_path = MODEL_PATH,
                  history_path = HISTORY_PATH,
                  save_every_epoch=True,
                  apply_weight_decay=False)

Epoch 1/30


2024-01-31 08:51:22.720827: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-31 08:51:25.794941: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb2ceb61e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 08:51:25.794962: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-01-31 08:51:25.798371: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706691085.871744 1159256 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


879/880 [============================>.] - ETA: 0s - loss: 0.4925 - sensitivity: 0.4284 - specificity: 0.9652 - accuracy: 0.9288INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/1--0.4923-0.4286-0.9652-0.6375-0.9630.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/1--0.4923-0.4286-0.9652-0.6375-0.9630.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/1--0.4923-0.4286-0.9652-0.6375-0.9630.pb
880/880 [==============================] - 137s 136ms/step - loss: 0.4923 - sensitivity: 0.4286 - specificity: 0.9652 - accuracy: 0.9289 - val_loss: 0.3524 - val_sensitivity: 0.6375 - val_specificity: 0.9630 - val_accuracy: 0.9414
Epoch 2/30
880/880 [==============================] - ETA: 0s - loss: 0.3370 - sensitivity: 0.6361 - specificity: 0.9686 - accuracy: 0.9467INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/2--0.3370-0.6361-0.9686-0.6917-0.9664.pb/assets


...

Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/28--0.2141-0.7625-0.9817-0.7611-0.9819.pb
880/880 [==============================] - 119s 136ms/step - loss: 0.2141 - sensitivity: 0.7625 - specificity: 0.9817 - accuracy: 0.9676 - val_loss: 0.2155 - val_sensitivity: 0.7611 - val_specificity: 0.9819 - val_accuracy: 0.9674
Epoch 29/30
880/880 [==============================] - ETA: 0s - loss: 0.2120 - sensitivity: 0.7640 - specificity: 0.9820 - accuracy: 0.9680INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/29--0.2120-0.7640-0.9820-0.7752-0.9817.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/29--0.2120-0.7640-0.9820-0.7752-0.9817.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/29--0.2120-0.7640-0.9820-0.7752-0.9817.pb
880/880 [==============================] - 122s 138ms/step - loss: 0.2120 - sensitivity: 0.7640 - specificity: 0.9820 - accuracy: 0.9680 - val_loss: 0.2083 - val_sensitivity: 0.7752 - val_specificity: 0.9817 - val_accuracy: 0.9684
Epoch 30/30
880/880 [==============================] - ETA: 0s - loss: 0.2114 - sensitivity: 0.7651 - specificity: 0.9820 - accuracy: 0.9681INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/30--0.2114-0.7651-0.9820-0.7727-0.9810.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/30--0.2114-0.7651-0.9820-0.7727-0.9810.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_sG_wo_decay.tf/30--0.2114-0.7651-0.9820-0.7727-0.9810.pb
880/880 [==============================] - 129s 147ms/step - loss: 0.2114 - sensitivity: 0.7651 - specificity: 0.9820 - accuracy: 0.9681 - val_loss: 0.2117 - val_sensitivity: 0.7727 - val_specificity: 0.9810 - val_accuracy: 0.9676
